In [ ]:
import org.apache.spark.streaming.StreamingContext._
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SQLContext
import org.apache.spark.streaming.Duration
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.types
import spark.implicits._

In [ ]:
!rm -r checkpoint


In [ ]:
!mkdir checkpoint

In [ ]:
!rm -r spark-warehouse

In [5]:
val df = spark.read
    .json("input/*.json")

df: org.apache.spark.sql.DataFrame = [agent-phone: string, bath: string ... 17 more fields]


In [6]:
df.count()

res0: Long = 34


In [7]:
val schema = df.schema

schema: org.apache.spark.sql.types.StructType = StructType(StructField(agent-phone,StringType,true),StructField(bath,StringType,true),StructField(bedrooms,StringType,true),StructField(beds,StringType,true),StructField(data-est,StringType,true),StructField(data-lk,StringType,true),StructField(data-pk,StringType,true),StructField(data-pos,StringType,true),StructField(full_bathrooms,StringType,true),StructField(home_type,StringType,true),StructField(ldp-description-text,StringType,true),StructField(ldp-phone-link,StringType,true),StructField(price,StringType,true),StructField(property-info-address-citystatezip,StringType,true),StructField(property-info-address-main,StringType,true),StructField(sold date,StringType,true),StructField(sqft,StringType,true),StructField(status,StringType,true),...


In [8]:
val df = spark.readStream
    .format("json")
    .schema(schema)
    .option("multiLine", "false")
    .option("cleanSource", "archive")
    .option("sourceArchiveDir", f"input_archive")
    .load("input/*.json")

df: org.apache.spark.sql.DataFrame = [agent-phone: string, bath: string ... 17 more fields]


In [9]:
/*
df
    .writeStream
    .outputMode("append")
    .option("truncate", false)
    .format("console")
    .start()
    .awaitTermination()
    */

In [10]:
val df2 = df.writeStream
    .option("path", "output")
    .option("checkpointLocation", "checkpoint")
    .trigger(org.apache.spark.sql.streaming.Trigger.ProcessingTime("5 seconds"))
    .toTable("houses")

df2: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@19e0a546


In [ ]:
df2.processAllAvailable
//df2.stop
df2.awaitTermination

In [ ]:
val df3 = spark.read.table("houses")

In [ ]:
df3.count

In [ ]:
df2.stop